# Autoencoder Variational
from https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/Variational_autoencoder.py

In [12]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [13]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [14]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [15]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [16]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize([0.5], [0.5])
])

In [17]:
# dataset = MNIST('./data', transform=img_transform)
# dataset = MNIST('D:/datasets_pytorch/MNIST30', download=True, transform=img_transform)
dataset = MNIST('D:/datasets_pytorch/MNIST30', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


In [19]:
model = VAE()
if torch.cuda.is_available():
    model.cuda()
print(model)

VAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
)


In [20]:
reconstruction_function = nn.MSELoss(size_average=False)

In [21]:
def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD

In [22]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [25]:

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

        './mlp_img/image_{}.png'
        
        
        
torch.save(model.state_dict(), './vae.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 37.792309
Train Epoch: 0 [12800/60000 (21%)]	Loss: 34.222244
Train Epoch: 0 [25600/60000 (43%)]	Loss: 34.204964
Train Epoch: 0 [38400/60000 (64%)]	Loss: 34.305058
Train Epoch: 0 [51200/60000 (85%)]	Loss: 33.684017
====> Epoch: 0 Average loss: 34.9055
Train Epoch: 1 [0/60000 (0%)]	Loss: 33.554298
Train Epoch: 1 [12800/60000 (21%)]	Loss: 32.643803
Train Epoch: 1 [25600/60000 (43%)]	Loss: 31.776985
Train Epoch: 1 [38400/60000 (64%)]	Loss: 32.852196
Train Epoch: 1 [51200/60000 (85%)]	Loss: 33.524193
====> Epoch: 1 Average loss: 33.1234
Train Epoch: 2 [0/60000 (0%)]	Loss: 31.596096
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.460777
Train Epoch: 2 [25600/60000 (43%)]	Loss: 33.719585
Train Epoch: 2 [38400/60000 (64%)]	Loss: 32.629463
Train Epoch: 2 [51200/60000 (85%)]	Loss: 32.169174
====> Epoch: 2 Average loss: 32.2782
Train Epoch: 3 [0/60000 (0%)]	Loss: 31.106237
Train Epoch: 3 [12800/60000 (21%)]	Loss: 32.367188
Train Epoch: 3 [25600/60000 (43%)]	Loss: 3

Train Epoch: 28 [25600/60000 (43%)]	Loss: 29.295286
Train Epoch: 28 [38400/60000 (64%)]	Loss: 29.486088
Train Epoch: 28 [51200/60000 (85%)]	Loss: 29.472881
====> Epoch: 28 Average loss: 29.3651
Train Epoch: 29 [0/60000 (0%)]	Loss: 29.985113
Train Epoch: 29 [12800/60000 (21%)]	Loss: 29.332310
Train Epoch: 29 [25600/60000 (43%)]	Loss: 29.737799
Train Epoch: 29 [38400/60000 (64%)]	Loss: 29.741058
Train Epoch: 29 [51200/60000 (85%)]	Loss: 30.804796
====> Epoch: 29 Average loss: 29.3319
Train Epoch: 30 [0/60000 (0%)]	Loss: 30.294518
Train Epoch: 30 [12800/60000 (21%)]	Loss: 28.708334
Train Epoch: 30 [25600/60000 (43%)]	Loss: 28.676201
Train Epoch: 30 [38400/60000 (64%)]	Loss: 30.260319
Train Epoch: 30 [51200/60000 (85%)]	Loss: 29.581778
====> Epoch: 30 Average loss: 29.2927
Train Epoch: 31 [0/60000 (0%)]	Loss: 28.887867
Train Epoch: 31 [12800/60000 (21%)]	Loss: 30.059639
Train Epoch: 31 [25600/60000 (43%)]	Loss: 28.583740
Train Epoch: 31 [38400/60000 (64%)]	Loss: 29.610472
Train Epoch: 31 [

Train Epoch: 56 [25600/60000 (43%)]	Loss: 30.142809
Train Epoch: 56 [38400/60000 (64%)]	Loss: 28.319077
Train Epoch: 56 [51200/60000 (85%)]	Loss: 27.808590
====> Epoch: 56 Average loss: 28.6028
Train Epoch: 57 [0/60000 (0%)]	Loss: 29.157114
Train Epoch: 57 [12800/60000 (21%)]	Loss: 28.021679
Train Epoch: 57 [25600/60000 (43%)]	Loss: 27.927971
Train Epoch: 57 [38400/60000 (64%)]	Loss: 28.373938
Train Epoch: 57 [51200/60000 (85%)]	Loss: 28.347799
====> Epoch: 57 Average loss: 28.6034
Train Epoch: 58 [0/60000 (0%)]	Loss: 28.949301
Train Epoch: 58 [12800/60000 (21%)]	Loss: 28.111734
Train Epoch: 58 [25600/60000 (43%)]	Loss: 30.014038
Train Epoch: 58 [38400/60000 (64%)]	Loss: 27.558256
Train Epoch: 58 [51200/60000 (85%)]	Loss: 27.618324
====> Epoch: 58 Average loss: 28.6164
Train Epoch: 59 [0/60000 (0%)]	Loss: 29.233082
Train Epoch: 59 [12800/60000 (21%)]	Loss: 28.122669
Train Epoch: 59 [25600/60000 (43%)]	Loss: 28.704840
Train Epoch: 59 [38400/60000 (64%)]	Loss: 29.233484
Train Epoch: 59 [

Train Epoch: 84 [25600/60000 (43%)]	Loss: 29.023607
Train Epoch: 84 [38400/60000 (64%)]	Loss: 29.205750
Train Epoch: 84 [51200/60000 (85%)]	Loss: 29.590260
====> Epoch: 84 Average loss: 28.3001
Train Epoch: 85 [0/60000 (0%)]	Loss: 28.075115
Train Epoch: 85 [12800/60000 (21%)]	Loss: 28.917814
Train Epoch: 85 [25600/60000 (43%)]	Loss: 29.151913
Train Epoch: 85 [38400/60000 (64%)]	Loss: 27.811110
Train Epoch: 85 [51200/60000 (85%)]	Loss: 27.974508
====> Epoch: 85 Average loss: 28.2826
Train Epoch: 86 [0/60000 (0%)]	Loss: 27.361759
Train Epoch: 86 [12800/60000 (21%)]	Loss: 28.702272
Train Epoch: 86 [25600/60000 (43%)]	Loss: 28.220060
Train Epoch: 86 [38400/60000 (64%)]	Loss: 27.506721
Train Epoch: 86 [51200/60000 (85%)]	Loss: 27.854393
====> Epoch: 86 Average loss: 28.2421
Train Epoch: 87 [0/60000 (0%)]	Loss: 26.357504
Train Epoch: 87 [12800/60000 (21%)]	Loss: 28.818518
Train Epoch: 87 [25600/60000 (43%)]	Loss: 29.362204
Train Epoch: 87 [38400/60000 (64%)]	Loss: 28.339544
Train Epoch: 87 [